In [1]:
from langchain_core.tools import tool

1) Creating tool with tool decorator

Creating a custom tool is a three step process

In [2]:
# Step 1 - create a function
def multiply(a,b):
    """Multiply two numbers"""
    return a*b

In [3]:
# Step 2 - add type hints
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [4]:
# Step 3 - add tool decorator
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [6]:
result = multiply.invoke({'a': 3, 'b': 5})
result

15

In [7]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
#tool schema is sent to LLM
multiply.args_schema.model_json_schema()

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

2) Using Structured Tool and Pydantic

In [12]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [15]:
class MultiplyInput(BaseModel):
    a: int=Field( description='First number to add')
    b: int=Field( description='The second number to add')

In [16]:
def multiply_func(a:int, b:int)->int:
    return a*b

In [17]:
multiply_tool = StructuredTool.from_function(
    func= multiply_func, 
    name='multiply', 
    description='Multiply two numbers', 
    args_schema= MultiplyInput
)

In [18]:
multiply_tool.invoke({'a': 3, 'b': 4})

12

3) Using BaseTool class

both @tool and StructuredTool are built on the top of BaseTool

In [19]:
from langchain_core.tools import BaseTool
from typing import Type

In [20]:
class MultiplyInput(BaseModel):
    a: int=Field( description='First number to add')
    b: int=Field( description='The second number to add')

In [21]:
class MultiplyTool(BaseTool):
    name: str = 'multiply'
    description: str = "Multiply two numbers"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a:int, b:int)->int:
        return a*b

In [22]:
multiply_tool = MultiplyTool()

In [23]:
print(multiply_tool.invoke({'a': 3, 'b':4}))
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

12
multiply
Multiply two numbers
{'a': {'description': 'First number to add', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'title': 'B', 'type': 'integer'}}
